In [155]:
### ADDED NUM_CLAIMS PARAMETER

def parse(driver,alias_list,genes,target_frequency,num_claims):
    results = []
    count = [0]*len(alias_list)

    #TITLE
    #titleStr = driver.find_element_by_id("title").text
    titleStr = driver.title.split(" - ")[1].strip()
    
    #print(titleStr)
    for aliases in alias_list:
        for cur in aliases:
            if len(str(cur))>=3:
                #print("entered")
                if str(cur) in titleStr:
                    countTitle=(titleStr.count(" " + str(cur) + " ")+titleStr.count(" " + str(cur) + ".")+\
                    titleStr.count(" " + str(cur) + ",")+titleStr.count(" " + str(cur) + "'")+\
                    titleStr.count(" " + str(cur) + "/")+titleStr.count("/" + str(cur) + " ")+\
                    titleStr.count(" " + str(cur) + ":")+titleStr.count("-" + str(cur) + " "))
                    if countTitle!=0:
                        results.append(aliases[0])
                        count[alias_list.index(aliases)]+=countTitle
                        break
                ## MAYBE ADD LENGTH RESTRICTION, len(str(cur))>5 and str(cur).lower() in titleStr
                elif str(cur).lower() in titleStr:
                    lower = str(cur).lower()
                    countTitle=(titleStr.count(" " + lower + " ")+titleStr.count(" " + lower + ".")+\
                    titleStr.count(" " + lower + ",")+titleStr.count(" " + lower + "'")+\
                    titleStr.count(" " + lower + "/")+titleStr.count("/" + lower + " ")+\
                    titleStr.count(" " + lower + ":")+titleStr.count("-" + lower + " "))
                    if countTitle!=0:
                        results.append(aliases[0])
                        count[alias_list.index(aliases)]+=countTitle
                        break
        
        
    #ABSTRACT
    abstractStr = driver.find_element_by_id("abstract").text

    if not abstractStr:
        abstractStr = driver.find_element_by_tag_name("patent-text").text
    #print(abstractStr)

    ## aliases - list of aliases for current gene
    ## cur - the current alias that is being looked for in the text
    for aliases in alias_list:
        for cur in aliases:
            #y=str(y).lower()
            if len(str(cur))>=3:
                #print("entered")
                if str(cur) in abstractStr:
                    countAbstract = 3*(abstractStr.count(" " + str(cur) + " ")+abstractStr.count(" " + str(cur) + ".")+\
                    abstractStr.count(" " + str(cur) + ",")+abstractStr.count(" " + str(cur) + "'")+\
                    abstractStr.count(" " + str(cur) + "/")+abstractStr.count("/" + str(cur) + " ")+\
                    abstractStr.count(" " + str(cur) + ":")+abstractStr.count("-" + str(cur) + " "))
                    count[alias_list.index(aliases)]+=countAbstract
                    #print(str(temp) + " gene: "+ str(y))
                elif str(cur).lower() in abstractStr:
                    lower = str(cur).lower()
                    countAbstract = 3*(abstractStr.count(" " + lower + " ")+abstractStr.count(" " + lower + ".")+\
                    abstractStr.count(" " + lower + ",")+abstractStr.count(" " + lower + "'")+\
                    abstractStr.count(" " + lower + "/")+abstractStr.count("/" + lower + " ")+\
                    abstractStr.count(" " + lower + ":")+abstractStr.count("-" + lower + " "))
                    count[alias_list.index(aliases)]+=countAbstract
        
        
    #CLAIMS
    claimList=[]
    
    claims = driver.find_elements_by_class_name("claim-text")
    count_claims = 0
    for x in claims:
        if count_claims>=num_claims:
            break
        claimList.append(x.text+"\n")
        count_claims+=1
        
    #print(claimList)    
    claimStr = ""
    for x in claimList:
        claimStr+=x

    #print(claimStr)
    for aliases in alias_list:
        for cur in aliases:  
            if len(str(cur))>=3:
                #print("entered")
                if str(cur) in claimStr:
                    count[alias_list.index(aliases)]+=claimStr.count(" " + str(cur) + " ")+claimStr.count(" " + str(cur) + ".")+\
                    claimStr.count(" " + str(cur) + ",")+claimStr.count(" " + str(cur) + "'")+\
                    claimStr.count(" " + str(cur) + "/")+claimStr.count("/" + str(cur) + " ")+\
                    claimStr.count(" " + str(cur) + ":")+claimStr.count("-" + str(cur) + " ")
                elif str(cur).lower() in claimStr:
                    lower = str(cur).lower()
                    count[alias_list.index(aliases)]+=claimStr.count(" " + lower + " ")+claimStr.count(" " + lower + ".")+\
                    claimStr.count(" " + lower + ",")+claimStr.count(" " + lower + "'")+\
                    claimStr.count(" " + lower + "/")+claimStr.count("/" + lower + " ")+\
                    claimStr.count(" " + lower + ":")+claimStr.count("-" + lower + " ")

    
    sort = sorted(zip(count, genes), reverse=True)
    count = 0
    r = []
    print_stuff=[]
    for x in sort:
        if count>=3:
            break
        elif x[1] not in results:
            r.append(x)
            print_stuff.append(x)
            count+=1
        elif x[1] in results:
            print_stuff.append(x)
    
    for x in r:
        if x[0]>=target_frequency:
            results.append(x[1])
        elif x[0]>0 and len(results)==0:
            results.append(x[1])
            break

    print(print_stuff)
    
    return results

In [1]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium import webdriver
import time
import xlrd
driver = webdriver.Firefox(executable_path=r'C:\Users\jjohn\Desktop\Scraping\geckodriver-v0.29.1-win64\geckodriver.exe') #make sure this exists somewhere in a local, varies from user to user, and copy the path here

driver.get("https://patents.google.com/patent/US20180010134A1/en?q=tp53&oq=tp53")

time.sleep(5)

#GENE ARRAY
l = newaliasesfromfile()
gene = genes()
#print(l)

list = parse(driver,l,gene,3,10)

print("Genes found")
for x in list:
    print(x)

driver.close()

KeyboardInterrupt: 

In [94]:
import xlrd
def aliasesfromfile():
    wb = xlrd.open_workbook  # read all genes from excel sheet
    ws = wb.sheet_by_index(0)
    x = len(ws.col_values(0))

    genelist = ws.col_values(0)
    genelist.pop(0)  # genelist = list of all target names
    aliases = []
    for dog in range(x):
        x = ws.row_values(dog)
        x[:] = [y for y in x if y]
        x = x[1:]
        x = str(x)
        aliases.append(x)
    return aliases

In [56]:
import pandas as pd
def newaliasesfromfile():
    pepega = pd.read_excel("Gene_Aliases_All.xlsx")
    #print(pepega["Unnamed: 0"].values[0])
    #print(pepega.values[0])
    poogers = []
    for dog in range(len(pepega.values)):
        peepo = []
        for cow in range(len(pepega.values[dog])-1):
            if pd.isnull(pepega.values[dog][cow]):
                break
            peepo.append(pepega.values[dog][cow])
        poogers.append(peepo)
    return poogers

In [67]:
import pandas as pd
def genes():
    pepega = pd.read_excel("Gene_Aliases_All.xlsx")
    #print(pepega["Unnamed: 0"].values[0])
    #print(pepega.values[0])
    return pepega["Unnamed: 0"]